# <center>TD3: arbres de décision avec *scikit-learn* </center>



Nous allons utiliser *scikit-learn* pour mettre en place des arbres de décision. Voici le lien [doc](https://scikit-learn.org/stable/modules/tree.html) vers la documentation de *scikit-learn* concernant les arbres de décision 

In [1]:
# Importation des packages
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn import tree
import numpy as np
import pandas as pd




ModuleNotFoundError: No module named 'sklearn'

### Première partie: utilisation de la classe *DecisionTreeClassifier*

Nous allons commencer à travailler avec le jeu de données *magasin.csv* que vous connaissez bien maintenant.
Déterminer un dataframe *X* qui contiendra toutes les variables caractéristiques et un dataframe *Y* la variable cible.

In [2]:
df_magasin = pd.read_csv('data/magasin.csv',sep=';')
Y=df_magasin.loc[:,'Achat']
print(Y)
X=df_magasin.iloc[:,1:]

print(X)


NameError: name 'pd' is not defined

En vous aidant de la documentation de *scikit-learn*, entrainer le modèle (construction de l'arbre de décision) et faire afficher l'arbre de décision construit

In [ ]:
# Initialisation des paramètres du modèle
model_tree = DecisionTreeClassifier()

# Entraînement du modèle d'arbre
model_tree.fit(X, Y)

# Visualisation de l'arbre
tree.plot_tree(model_tree)
plt.show()




Vous pouvez à l'aide du code ci-dessous avoir une meilleure représentation de l'arbre.
Comparer l'arbre obtenu à celui que vous aviez construit avec votre programme écrit à la main. Il y a des différences.
En étudiant dans la documentation, les paramètres que l'on a la possibilité  d'utiliser pour construire l'arbre avec *DecisionTreeClassifier*, essayez de voir quelques différences.   

> Eh oui, on est loin d'avoir implémenter complètement cet algorithme, il y a de nombreux critères à prendre en compte pour pouvoir optimiser l'apprentissage.

In [ ]:
import graphviz
columns = X.columns[np.logical_not(X.columns == "Achat")]


dot_data = tree.export_graphviz(model_tree, out_file=None,
                                feature_names=columns.to_list(),
                                class_names=['non achat','achat'],
                                filled=True, rounded=True,
                                special_characters=True)
graph = graphviz.Source(dot_data)
#graph.render("magasin")
graph

### Seconde partie: optimisation du modèle 
 
> dans le monde de l'apprentissage automatique, on adore les néologismes issus de la langue anglaise => vous verrez souvent ce terme: *fine-tuner* le modèle. 

Le but sera de tester différentes valeurs des hyperparamètres du modèle pour l'optimiser. On utilisera pour cela la technique de la validation croisée comme dans le TD précédent.
Nous utiliserons le jeu de données *iris* qui est plus volumineux que le premier utilisé. 

- charger le jeu de données *iris*
- déterminer un ensemble d'apprentissage et un ensemble de test (60% apprentissage et 40% test). Dans la méthode que vous utiliserez pour construire ces deux ensembles, intéressez vous au paramètre *random_state* et passez lui la valeur maximale suggérée.
> Le jeu de données *iris* est un jeu de données très simple dont le modèle est assez facile à déterminé. Si l'on veut voir une différence après la validation croisée sur les résultats de prédiction, il faut rendre l'apprentissage plus difficile d'où la répartition ensemble d'apprentissage et ensemble de test choisi.

- construire un arbre de décision relatif à ce jeu de donnée et faire afficher l'arbre
- calculer le score obtenu sur l'ensemble de test (exactitude)

> lancer plusieurs fois votre traitement (10 fois). Qu'observez vous par rapport à la valeur de l'exactitude de la prédiction sur l'ensemble de test ?

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import tree

iris = load_iris()
X, Y = iris.data, iris.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.6, random_state=42)
arbre_decision = tree.DecisionTreeClassifier()
arbre_decision.fit(X_train, Y_train)
tree.plot_tree(arbre_decision, filled=True)
arbre_decision.score(X_test,Y_test)

Nous allons maintenant étudier 2 hyperparamètres du modèle: *max_depth* et *min_samples_leaf*

- regarder dans la documentation en quoi ces deux hyperparamètres influencent la construction de l'arbre. Vous pouvez tester à la main plusieurs de leurs valeurs pour visualiser les conséquences de leur modification sur le modèle obtenu.
- déterminer à l'aide de la technique de la validation croisée, la meilleure valeur pour les 2 hyperparamètres précédents.
Vous utiliserai la classe *GridSearchCV* qui permet de réaliser tous les tests avec les hyperparamètres concernés et vous donnera les valeurs de ceux-ci optimisés pour maximiser l'exactitude
[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
et [exemple](https://www.mygreatlearning.com/blog/gridsearchcv/)


In [ ]:
from sklearn import model_selection
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(iris.data, iris.target,
    test_size=0.4, random_state=42)
from sklearn.model_selection import GridSearchCV
parametre_grid = {"max_depth": [1, 2, 3, 4, 5, 6, 7],
      "min_samples_split": [2, 3, 5, 10, 15, 20]}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid=parametre_grid, cv=5)
grid_search.fit(X_train, Y_train)

print("\n The best parameters across ALL searched params:\n",grid_search.best_params_)

> Attention à l'interprétation: ce sont les meilleures valeurs des hyperparamètres de notre modèle pour le jeu de données que nous utilisons et avec la séparation que nous avons mis en place (60% apprentissage et 40% test)

Utiliser les valeurs des paramètres déterminées précédemment sur la partition initiale des données (60% apprentissage et 40% test). Que constatez vous ?

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import tree

iris = load_iris()
X, Y = iris.data, iris.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.60, random_state=42)
arbre_decision = tree.DecisionTreeClassifier(max_depth=4, min_samples_split= 2)
arbre_decision.fit(X_train, Y_train)
tree.plot_tree(arbre_decision, filled=True)
# moyenne de l'accuracy
arbre_decision.score(X_test,Y_test)


### Un autre cas à étudier


Utiliser maintenant le jeu de données relatif au Titanic. Nous utiliserons les colonnes qui sont notées dans la cellule suivante de ce notebook.

- séparer les données en 2 ensembles: aprentissage => 80% et test=> 20%
- construire l'arbre de décision
- calculer le score obtenu sur l'ensemble de test
- optimiser les hyperparamètres
- afficher le nouveau score obtenu et les valeurs des hyperparamètres déterminées par la phase d'optimisation (les hyperparamètres à optimiser sont donnés dans la cellule suivante de ce notebook)

> attention: afin d'utiliser la classe *DecisionTreeClassifier*,  il faut que:
- toutes les valeurs liées aux caractéristiques du jeu de données soient des nombres
- qu'aucune valeur du jeu de données ne soit nulle (vide)  
*Pour la colonne *Age* vous remplacerez les valeurs nulles par la moyenne de l'âge des passagers.
Pour les valeurs nulles dans les autres colonnes, vous supprimerez la ligne du jeu de données qui correspond à cette valeur nulle.
Il faudra donc mettre en place un pré-traitement sur les données avant de construire l'arbre.*

In [ ]:
from sklearn.model_selection import GridSearchCV

# Charger le jeu de données complet
df = pd.read_csv('data/titanic.csv', sep='\t' )

# Prétraitement des données (nous allons utiliser uniquement ces colonnes)
selected_columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked' ,'Survived']
df = df[selected_columns]

# Gérer les valeurs manquantes (remplacer les valeurs manquantes de 'age' par la moyenne)
df['Age'].fillna(df['Age'].mean(), inplace=True)

df = df.dropna()

# Convertir la colonne 'Sex' en une colonne binaire (0 pour male, 1 pour female)
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

df['Embarked']=df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# Découper le jeu de données en deux parties => X et Y:
# la première partie X correspond aux caractéristiques (donc toutes les colonnes sauf 'Survived')
# la seconde partie Y ne comprend que les données qui correspondent à la classe (la cible) qui ici est constituée de la la colonne 'Survived'
X = df.drop('Survived', axis=1)  # Les caractéristiques
Y = df['Survived']              # La variable cible (Survived)

# Division en 80% pour l'apprentissage et 20% pour le test (regarder la fonction: train_test_split(...))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

arbre_decision = tree.DecisionTreeClassifier()
arbre_decision.fit(X_train, Y_train)

print(arbre_decision.score(X_test,Y_test))

params = {
    'criterion':  ['gini', 'entropy'],
    'max_depth':  [None, 2, 4, 6, 8, 10],
    'max_features': [None, 'sqrt', 'log2', 0.2, 0.4, 0.6, 0.8],
    'splitter': ['best', 'random']
}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid=params, cv=5)
grid_search.fit(X_train, Y_train)
print(grid_search.best_estimator_.score(X_test, Y_test))
print("\n The best parameters across ALL searched params:\n",grid_search.best_params_)



### En Résumé:

Les arbres de décision ont plusieurs avantages qui les rendent intéressants dans des contextes où il est utile de comprendre la séquence de décisions prise par le modèle :

- Ils sont simples à comprendre et à visualiser.

- Ils nécessitent peu de préparation des données (pas de normalisation, etc.).

- Le coût d’utilisation des arbres est logarithmique.

- Ils sont capables d’utiliser des données catégorielles et numériques.

- Ils sont capables de traiter des problèmes multi-classe.

- Le résultat est directement visualisable.

**Ces modèles présentent néanmoins deux désavantages majeurs :**

- Sur-apprentissage : parfois les arbres générés sont trop complexes et généralisent mal. Choisir des bonnes valeurs pour les paramètres profondeur maximale (max_depth) et nombre minimal d’exemples par feuille (min_samples_leaf) permet d’éviter ce problème.

- Il peut arriver que les arbres générés ne soient pas équilibrés (ce qui implique que le temps de parcours n’est plus logarithmique). Il est donc recommandé d’ajuster le jeu de données avant la construction, pour éviter qu’une classe domine largement les autres (en termes de nombre d’exemples d’apprentissage).

